<a href="https://colab.research.google.com/github/atharvgarg/financial_news_summary/blob/main/Optimal_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 35.3 MB/s 
     |████████████████████████████████| 365 kB 65.4 MB/s 
     |████████████████████████████████| 1.3 MB 54.7 MB/s 
     |████████████████████████████████| 6.6 MB 52.6 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 596 kB 69.5 MB/s 
     |████████████████████████████████| 141 kB 65.7 MB/s 
     |████████████████████████████████| 115 kB 70.4 MB/s 
     |████████████████████████████████| 212 kB 70.4 MB/s 
     |████████████████████████████████| 127 kB 56.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!git clone https://github.com/nadasalah99/bert-extractive-summarizer

Cloning into 'bert-extractive-summarizer'...
remote: Enumerating objects: 276, done.
remote: Total 276 (delta 0), reused 0 (delta 0), pack-reused 276
Receiving objects: 100% (276/276), 57.15 KiB | 14.29 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [7]:
!pip install -r /content/bert-extractive-summarizer/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.3 MB 279 kB/s 
     |████████████████████████████████| 753.4 MB 7.1 kB/s 
     |████████████████████████████████| 10.6 MB 28.7 MB/s 
     |████████████████████████████████| 387 kB 17.0 MB/s 
     |████████████████████████████████| 2.1 MB 48.4 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 286 kB 54.6 MB/s 
     |████████████████████████████████| 7.1 MB 56.5 MB/s 
     |████████████████████████████████| 248 kB 66.9 MB/s 
     |████████████████████████████████| 576 kB 64.2 MB/s 
     |████████████████████████████████| 132 kB 50.2 MB/s 
     |████████████████████████████████| 880 kB 68.2 MB/s 
     |████████████████████████████████| 2.2 MB 55.1 MB/s 
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO:

In [5]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install spacy~=2.2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-2.2.4-cp37-cp37m-manylinux1_x86_64.whl (10.6 MB)
  Using cached plac-1.1.3-py2.py3-none-any.whl (20 kB)
     |████████████████████████████████| 3.7 MB 38.2 MB/s 
     |████████████████████████████████| 184 kB 59.3 MB/s 
  Using cached thinc-7.4.0-cp37-cp37m-manylinux1_x86_64.whl (2.2 MB)
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.4
    Uninstalling srsly-2.4.4:
      Successfully uninstalled srsly-2.4.4
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: blis
    Found existing installation: blis 0.7.8
    Uninstalling blis-0.7.8:
      Successfully uninstalled blis-0.7.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0

# Import libraries

In [6]:
import pandas as pd
import numpy as np
import pickle
import re
import os
import string
import heapq

from transformers import AutoTokenizer, AutoModel, AutoModelForTokenClassification, AutoModelForSeq2SeqLM, pipeline

import torch

import logging

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing,metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
st = PorterStemmer()
nltk.download('stopwords')
stop=stopwords.words('english')

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Text Classification

In [9]:
def clean(text):
  #remove hashtags
  text = re.sub("#", " ", text)
  #remove all numbers 
  text = re.sub(r'\[0-9]*\]',' ',text)
  #remove duplicated chars
  text = re.sub(r'(.)\1+', r'\1', text)
  #remove :) or :(
  text = text.replace(':)', "")
  text = text.replace(':(', "")
  #remove multiple exclamation
  text = re.sub(r"(\!)\1+", ' ', text)
  #remove multiple question marks
  text = re.sub(r"(\?)\1+", ' ', text)
  #remove multistop
  text = re.sub(r"(\.)\1+", ' ', text)
  #remove additional spaces
  text = re.sub(r"[\s]+", " ", text)
  text = re.sub(r"[\n]+", " ", text)
  
  return text

def remStopWords(Text):
  return " ".join(word for word in Text.split() if word not in stop)

def stemWords(Text):
  return " ".join(st.stem(word) for word in Text.split())
  
def pipeline(Text):
  #preprocessing step
  Text = clean(Text)
  Text = "".join([char for char in Text if char not in string.ascii_letters]).strip()
  Text = remStopWords(str(Text))
  Text = stemWords(Text)
  #vectorize the text
  Text_Vector = tfidf_vect.transform([Text])
  predictions = clf_svm.predict(Text_Vector)
  return encoder.inverse_transform(predictions)[0]

# Extractive Model

In [10]:
cd /content/bert-extractive-summarizer

/content/bert-extractive-summarizer


In [11]:
ls

arabert_summarize_example.py  Makefile                  setup.cfg
Dockerfile                    README.md                 setup.py
Dockerfile.service            requirements-service.txt  summarize.py
example_file.txt              requirements.txt          summarizer/
LICENSE                       server.py                 tests/


In [11]:
from summarizer import Summarizer 

extractive_Model = AutoModel.from_pretrained('bert-base-uncased')
extractive_Tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

extractiveModelSummarizer = Summarizer(custom_model=extractive_Model, custom_tokenizer=extractive_Tokenizer)

https://huggingface.co/bert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpn_bj2ikx


storing https://huggingface.co/bert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
creating metadata file for /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "h

storing https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
creating metadata file for /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 

storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob":

storing https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpmtda9yyh


storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://hugg

## Model to Pickle

In [13]:
pickle.dump(extractiveModelSummarizer, open("/content/drive/MyDrive/Nlp/Pickles/extractiveModel.pkl", "wb"))  

# Abstractive Model

In [13]:
model_name = "Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old"
abstractive_Tokenizer = AutoTokenizer.from_pretrained(model_name)
abstractive_Model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp1c_s236j


storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/4aef3fc730005e8f1645715e0bf48f660dea26701ac359129d876e3233789467.d24951e82532aeeaee7ab02281590ea82c08b4dcc773db0c892dee877cd533cd
creating metadata file for /root/.cache/huggingface/transformers/4aef3fc730005e8f1645715e0bf48f660dea26701ac359129d876e3233789467.d24951e82532aeeaee7ab02281590ea82c08b4dcc773db0c892dee877cd533cd
https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp_w0timkr


storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/cabd85f7cba6b92e9fb7b2406e0b8da2da2a746bdcf1c980e1b59e984786c565.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/cabd85f7cba6b92e9fb7b2406e0b8da2da2a746bdcf1c980e1b59e984786c565.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpat6beb1i


storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/c61417a68695519ccbdcf9599b3377624fefc4633bb964f3289cf4b4593a4a38.c32c4a7596b5bfd7275f3afee64968a94b9951235630e31c699108573b683baf
creating metadata file for /root/.cache/huggingface/transformers/c61417a68695519ccbdcf9599b3377624fefc4633bb964f3289cf4b4593a4a38.c32c4a7596b5bfd7275f3afee64968a94b9951235630e31c699108573b683baf
https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp965vsu8f


storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/59b9c5d5f0c8316c2b56431c1c78c97caf104f2911b2628445c5a15540888cf0.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
creating metadata file for /root/.cache/huggingface/transformers/59b9c5d5f0c8316c2b56431c1c78c97caf104f2911b2628445c5a15540888cf0.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
loading file https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/cabd85f7cba6b92e9fb7b2406e0b8da2da2a746bdcf1c980e1b59e984786c565.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news

storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/c147149d80d9bf9f3471c6b017caef96326ba1fa4359c1cd7433fcfc4fc9eb94.320d20ca4053e17fa9777800a62434d5f9654b9adfea417539328eca86250159
creating metadata file for /root/.cache/huggingface/transformers/c147149d80d9bf9f3471c6b017caef96326ba1fa4359c1cd7433fcfc4fc9eb94.320d20ca4053e17fa9777800a62434d5f9654b9adfea417539328eca86250159
loading configuration file https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c147149d80d9bf9f3471c6b017caef96326ba1fa4359c1cd7433fcfc4fc9eb94.320d20ca4053e17fa9777800a62434d5f9654b9adfea417539328eca86250159
Model config EncoderDecoderConfig {
  "_name_or_path": "Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summariz

storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/9b0b934276ba3160f6c5f2302d5abc8519738b8688a3e2ddac6d695bb8be6043.fafdc0774d398784fa45105017064d8b4c5fd020b63bc1fc0251de8108826e53
creating metadata file for /root/.cache/huggingface/transformers/9b0b934276ba3160f6c5f2302d5abc8519738b8688a3e2ddac6d695bb8be6043.fafdc0774d398784fa45105017064d8b4c5fd020b63bc1fc0251de8108826e53
loading weights file https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9b0b934276ba3160f6c5f2302d5abc8519738b8688a3e2ddac6d695bb8be6043.fafdc0774d398784fa45105017064d8b4c5fd020b63bc1fc0251de8108826e53
All model checkpoint weights were used when initializing EncoderDecoderModel.

All the weights of EncoderDecoderMode

In [14]:
def abstractiveModelSummarizer(text):
  input_ids = abstractive_Tokenizer(
      [(text)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=512
  )["input_ids"]

  output_ids = abstractive_Model.generate(
      input_ids=input_ids,
      max_length=512,
      no_repeat_ngram_size=4,
      num_beams=4
  )[0]

  summary = abstractive_Tokenizer.decode(
      output_ids,
      skip_special_tokens=True,
      clean_up_tokenization_spaces=False
  )
  return summary

## Model to Pickle

In [22]:
pickle.dump(abstractiveModelSummarizer, open("/content/drive/MyDrive/Nlp/Pickles/abstractiveModelbbc.pkl", "wb"))  

## Model to Pickle

# Load Pickles

In [16]:
# for reading also binary mode is important
#clf_svm = pickle.load(open('/content/drive/MyDrive/Nlp/Pickles/TextClassifier.pkl', 'rb'))
  
encoder = pickle.load(open('/content/drive/MyDrive/Nlp/Pickles/LabelEncoder.pkl', 'rb'))

#tfidf_vect = pickle.load(open('/content/drive/MyDrive/Nlp/Pickles/TextVectorizer.pkl', 'rb'))

extractiveModel = pickle.load(open('/content/drive/MyDrive/Nlp/Pickles/extractiveModel.pkl', 'rb'))

abstractiveModel = pickle.load(open('/content/drive/MyDrive/Nlp/Pickles/abstractiveModelbbc.pkl', 'rb'))

#NER_tokenizer = pickle.load(open("/content/drive/MyDrive/Nlp/Pickles/NERTokenizer.pkl", 'rb'))

#NER_Model = pickle.load(open("/content/drive/MyDrive/Nlp/Pickles/NERModel.pkl", 'rb'))

# The Models Output

In [18]:
the_result_extractive = extractiveModel(textInput)
print(the_result_extractive)

Stranger Things is an American science fiction horror drama 
television series created by the Duffer Brothers, who also serve as 
showrunners and executive producers. The first season of the series 
was released on Netflix on July 15, 2016. The series has been critically acclaimed
for its characterization, atmosphere, acting, soundtrack, directing, writing,
and homages to 1980s films.


In [19]:
the_result_abstractive = abstractiveModel(textInput)  
print(the_result_abstractive)

the first season of the series was released on netflix on july 15, 2016. the duffer brothers developed stranger things as a mix of investigative drama and supernatural elements portrayed with horror, science fiction and childlike sensibilities. set in the 1980s, primarily in the fictional town of hawkins, indiana, stranger things centers on a number of mysteries and supernatural events occurring around the town and their impact on an ensemble of child and adult characters. one of netflix's flagship series, stranger things has attracted record viewership on the streaming platform. it has received numerous awards and nominations, including nominations from the golden globe awards, british academy television awards, directors guild of america awards, and grammy awards, as well as wins from the primetime emmy awards, american film institute, critics'choice awards, and people's choice awards.


In [20]:
the_result_hyrbid = abstractiveModel(the_result_extractive)
print(the_result_hyrbid)

stranger things is an american science fiction horror drama television series created by the duffer brothers, who also serve as showrunners and executive producers. the first season of the series was released on netflix on july 15, 2016. the series has been critically acclaimed for its depiction, atmosphere, acting, directing, writing, and homages to 1980s films.


In [21]:
!pip install rouge_score
from datasets import load_metric
rouge_score = load_metric('rouge')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=8a33c5230b7cb91de45f1e5f4e74903add193a6357b35ce00e13ac9a43b6eb6c
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [94]:
from datasets import load_dataset
data_files = {"train": "/content/testset.csv" , "test": "/content/testset.csv"}
news_dataset = load_dataset("csv", data_files = data_files)
news_dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3aff455ab19b3390/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'File_path', 'Articles', 'Summaries', 'summaries'],
        num_rows: 5
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'File_path', 'Articles', 'Summaries', 'summaries'],
        num_rows: 5
    })
})

In [27]:
news_dataset['test'].shard(num_shards=10, index=0)

Dataset({
    features: ['Unnamed: 0', 'Unnamed: 0.1', 'File_path', 'Articles', 'Summaries', 'Sumy'],
    num_rows: 45
})

In [95]:
def evaluate_extracted_summary(dataset, metric):
    summaries = [extractiveModel(text) for text in dataset["Articles"]]
    return metric.compute(predictions=summaries, references=dataset["Summaries"])

In [96]:
import pandas as pd

score = evaluate_extracted_summary(news_dataset["test"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL" ,"rougeLsum"]
rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 56.79, 'rouge2': 46.75, 'rougeL': 39.26, 'rougeLsum': 38.9}

In [97]:
def evaluate_abstracted_summary(dataset, metric):
    summaries = [abstractiveModel(text) for text in dataset["Articles"]]
    return metric.compute(predictions=summaries, references=dataset["Summaries"])

In [98]:
import pandas as pd

score = evaluate_abstracted_summary(news_dataset["test"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL" ,"rougeLsum"]
rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 73.99, 'rouge2': 66.15, 'rougeL': 52.32, 'rougeLsum': 52.32}

In [101]:
def evaluate_hybrid_summary(dataset, metric):
    summaries = [abstractiveModel(text) for text in dataset["summaries"]]
    return metric.compute(predictions=summaries, references=dataset["Summaries"])

In [102]:
import pandas as pd

score = evaluate_hybrid_summary(news_dataset["test"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL" ,"rougeLsum"]
rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': 46.52, 'rouge2': 38.41, 'rougeL': 34.98, 'rougeLsum': 34.92}